In [ ]:
# Step 1: Install dependencies
!pip install -q torch transformers datasets peft accelerate bitsandbytes evaluate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import math # Import math for perplexity calculation
from torch.utils.data import DataLoader
from peft import get_peft_model, LoraConfig
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00


In [ ]:
# Step 2: Load Model and Tokenizer
model_name = "Qwen/Qwen2.5-14B-Instruct"


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # ✅ Critical fix: Specify pad_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,  # ✅ Save memory
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00008.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# Step 3: Load and Prepare Dataset
dataset = load_dataset("tatsu-lab/alpaca")["train"]

# Construct forget and retain sets
forget_indices = [0, 1, 2, 3, 4]
D_forget = dataset.select(forget_indices)
D_retain = dataset.filter(lambda e, i: i not in forget_indices, with_indices=True)

In [ ]:
# Step 4: Preprocessing Function
def preprocess(examples):
    """Tokenize the instruction field only."""
    tokens = tokenizer(
        examples["instruction"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )
    return tokens  # ✅ Keep the standard tokenizer output structure

In [ ]:
# Step 5: Tokenize Datasets and Create DataLoaders
D_forget_tokenized = D_forget.map(preprocess, batched=True, remove_columns=D_forget.column_names)
D_forget_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

D_retain_tokenized = D_retain.map(preprocess, batched=True, remove_columns=D_retain.column_names)
D_retain_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

forget_loader = DataLoader(D_forget_tokenized, batch_size=1)
retain_loader = DataLoader(D_retain_tokenized, batch_size=8)


In [ ]:
# Step 6: Setup PEFT (LoRA)
config = LoraConfig(
    task_type="CAUSAL_LM",
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 14,776,325,120 || trainable%: 0.0426


In [ ]:
lambda_forget = 0.1      # 更轻的遗忘力度
retain_steps = 50        # 延长恢复训练
lr = 2e-6                 # 再降学习率
max_grad_norm = 0.3       # 更强梯度裁剪
num_epochs = 1

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
model.train()

for epoch in range(num_epochs):
    print(f"=== Epoch {epoch} ===")

    # 🧹 Ultra-soft unlearning
    for step, batch in enumerate(tqdm(forget_loader, desc="🧹 Unlearning phase")):
        batch = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss

        scaled_loss = -lambda_forget * loss
        scaled_loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        optimizer.zero_grad()

    # 📚 Recovery phase
    """    for step, batch in enumerate(tqdm(retain_loader, desc="📚 Retain phase")):
        if step >= retain_steps:
            break
        batch = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**batch, labels=batch["input_ids"])
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        optimizer.zero_grad() """


=== Epoch 0 ===


🧹 Unlearning phase: 100%|██████████| 5/5 [00:05<00:00,  1.01s/it]


In [ ]:
# Step 8: Save the Unlearned Adapter
model.save_pretrained("qwen_unlearned_adapter")
print("✅ LoRA adapter saved to ./qwen_unlearned_adapter")

✅ LoRA adapter saved to ./qwen_unlearned_adapter


In [ ]:
print("Starting evaluation...")
model.eval()

def calculate_perplexity(model, loader, n_batches=20):
    total_loss = 0
    num_batches = 0

    with torch.no_grad():
        effective_total = min(len(loader), n_batches)

        for batch in tqdm(loader, desc="Evaluating", total=effective_total):
            if num_batches >= n_batches:
                break

            batch = {k: v.to(model.device) for k, v in batch.items()}

            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                labels=batch["input_ids"]
            )

            total_loss += outputs.loss.item()
            num_batches += 1

    if num_batches == 0:
        return 0.0

    avg_loss = total_loss / num_batches
    perplexity = math.exp(avg_loss)
    return perplexity

forget_perp = calculate_perplexity(model, forget_loader, n_batches=20)
retain_perp = calculate_perplexity(model, retain_loader, n_batches=20)

print("\n--- Evaluation Results ---")
print(f"Forgetting set perplexity (expected ↑): {forget_perp:.2f}")
print(f"Retain set perplexity (expected ↓ or ~): {retain_perp:.2f}")

Starting evaluation...


Evaluating: 100%|██████████| 20/20 [01:03<00:00,  3.15s/it]


--- Evaluation Results ---
Forgetting set perplexity (expected ↑): 290198792.20
Retain set perplexity (expected ↓ or ~): 127631027.84
